In [2]:
from fastai import *
from fastai.tabular import *
from fastai.tabular.all import *
from fastai.collab import *
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import StratifiedKFold

path = Path('data/')

In [3]:
print(path.ls())

[Path('data/.ipynb_checkpoints'), Path('data/KNN_test.csv'), Path('data/KNN_train.csv'), Path('data/test.csv'), Path('data/test_dropped_290_without_nans.csv'), Path('data/test_dropped_290_with_nans.csv'), Path('data/train.csv'), Path('data/train_dropped_290_without_nans.csv'), Path('data/train_dropped_290_with_nans.csv'), Path('data/train_fastai.csv'), Path('data/X_test_imp.csv'), Path('data/X_train_imp.csv'), Path('data/y_train.csv')]


In [4]:
df = pd.read_csv(path/'train_dropped_290_without_nans.csv')
test = pd.read_csv(path/'test_dropped_290_without_nans.csv')

print(df.shape, test.shape)

(2838, 293) (939, 293)


In [5]:
y_train=pd.read_csv(path/'y_train.csv')['Culture']
df['Culture']=y_train

In [6]:
CAT_NAMES = ['0','1','2']
CONT_NAMES = list((df.drop(['0','1','2','Culture'],axis=1)).columns)
TARGET='Culture'

In [7]:
df[TARGET] = df[TARGET].astype('category')
for i in CAT_NAMES:
    df[i]=df[i].astype('category')
    test[i]=test[i].astype('category')

In [8]:
df

,0,1,2,3,4,5,6,7,8,9,...,284,285,286,287,288,289,290,291,292,Culture
0,2018.0,517.0,48.64,0.5369,0.5371,0.5375,0.5379,0.5384,0.5390,0.5397,...,0.3298,0.3317,0.3336,0.3355,0.3374,0.3393,0.3412,0.3431,0.3450,4
1,2019.0,1395.0,80.41,0.1159,0.1096,0.1034,0.0971,0.0909,0.0846,0.0783,...,0.3646,0.3595,0.3546,0.3501,0.3461,0.3426,0.3397,0.3375,0.3360,17
2,2017.0,44.0,43.64,-0.0204,-0.0207,-0.0210,-0.0212,-0.0215,-0.0218,-0.0220,...,0.3881,0.3964,0.4015,0.4040,0.4054,0.4059,0.4057,0.4052,0.4045,2
3,2018.0,1591.0,79.34,0.4827,0.4576,0.4324,0.4073,0.3821,0.3570,0.3318,...,0.2148,0.2147,0.2145,0.2143,0.2141,0.2140,0.2138,0.2136,0.2135,4
4,2017.0,681.0,144.76,-0.0202,-0.0191,-0.0181,-0.0170,-0.0160,-0.0150,-0.0140,...,0.2801,0.2795,0.2807,0.2838,0.2883,0.2937,0.2996,0.3055,0.3108,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2833,2019.0,1631.0,117.04,0.4429,0.4430,0.4430,0.4431,0.4431,0.4431,0.4431,...,0.4810,0.4850,0.4873,0.4885,0.4891,0.4892,0.4892,0.4894,0.4900,4
2834,2016.0,336.0,70.61,0.0620,0.0687,0.0755,0.0822,0.0890,0.0957,0.1025,...,0.4002,0.4048,0.4094,0.4140,0.4187,0.4233,0.4279,0.4325,0.4371,4
2835,2016.0,371.0,60.26,0.0460,0.0399,0.0338,0.0277,0.0216,0.0154,0.0093,...,0.2868,0.2873,0.2879,0.2885,0.2891,0.2898,0.2905,0.2912,0.2919,4
2836,2017.0,115.0,237.86,-0.0148,-0.0149,-0.0150,-0.0151,-0.0153,-0.0154,-0.0155,...,0.2688,0.2757,0.2820,0.2877,0.2933,0.2989,0.3043,0.3096,0.3149,5


In [9]:
procs = [Categorify, Normalize]

In [10]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [42]:
dls = TabularDataLoaders.from_df(df, 
                                 y_names=TARGET, 
                                 cat_names=CAT_NAMES,
                                 cont_names=CONT_NAMES,
                                 procs=procs,
                                 splits=splits)

In [43]:
dls.show_batch()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,Culture
0,2018,778,73.59,0.2986,0.2955,0.2924,0.2895,0.2867,0.2841,0.2816,0.2794,0.2775,0.2758,0.2745,0.2733,0.2722,0.2710,0.2698,0.2687,0.2675,0.2663,0.2651,0.2640,0.2628,0.2617,0.2605,0.2594,0.2582,0.2571,0.2560,0.2549,0.2538,0.2527,0.2516,0.2505,0.2495,0.2484,0.2474,0.2464,0.2454,0.2444,0.2435,0.2425,0.2416,0.2407,0.2398,0.2390,0.2382,0.2373,0.2366,0.2358,0.2351,0.2344,2.337000e-01,0.2331,0.2324,0.2319,0.2313,0.2308,0.2303,0.2298,0.2294,0.2290,0.2287,0.2284,0.2281,0.2279,0.2277,0.2275,0.2274,0.2273,0.2273,0.2273,0.2274,0.2275,0.2276,0.2278,0.2279,0.2281,0.2283,0.2285,0.2287,0.2289,0.2292,0.2294,0.2297,0.2299,0.2302,0.2305,0.2308,0.2311,0.2314,0.2317,0.2320,0.2323,0.2327,0.2331,2.335000e-01,0.2339,0.2344,0.2348,0.2353,0.2358,0.2363,0.2369,0.2374,0.2380,0.2387,0.2396,0.2408,0.2422,0.2437,0.2455,0.2474,0.2495,0.2516,0.2540,0.2564,0.2589,0.2615,0.2641,0.2668,0.2695,0.2724,0.2756,0.2792,0.2830,0.2871,0.2914,0.2960,0.3008,0.3057,0.3108,0.3161,0.3215,0.3270,0.3325,0.3382,0.3438,0.3495,0.3554,0.3619,0.3688,0.3761,0.3840,0.3923,0.4010,0.4102,0.4197,0.4297,0.4401,0.4509,0.4620,0.4751,0.4912,0.5094,0.5288,0.5487,0.5681,0.5862,0.6021,0.6150,0.6257,0.6357,0.6450,0.6536,0.6615,0.6686,0.6750,0.6809,0.6867,0.6922,0.6975,0.7023,0.7067,0.7105,0.7140,0.7175,0.7208,0.7238,0.7265,0.7288,0.7307,0.7320,0.7327,0.7327,0.7320,0.7308,0.7295,0.7281,0.7266,0.7250,0.7232,0.7214,0.7194,0.7174,0.7152,0.7129,0.7105,0.7080,0.7056,0.7031,0.7007,0.6982,0.6958,0.6937,0.6924,0.6917,0.6912,0.6909,0.6905,0.6899,0.6888,0.6870,0.6844,0.6807,0.6758,0.6694,0.6613,0.6500,0.6357,0.6215,0.6080,0.5940,0.5796,0.5651,0.5504,0.5356,0.5209,0.5064,0.4922,0.4783,0.4650,0.4522,0.4401,0.4288,0.4184,0.4090,0.4040,0.4016,0.3978,0.3941,0.3904,0.3867,0.3829,0.3792,0.3755,0.3718,0.3680,0.3643,0.3606,0.3569,0.3531,0.3494,0.3457,0.3420,0.3382,0.3345,0.3302,0.3251,0.3193,0.3132,0.3070,0.3012,0.2959,0.2914,0.2880,0.2855,0.2835,0.2818,0.2805,0.2795,0.2788,0.2783,0.2780,0.2779,0.2779,0.2779,0.2780,0.2781,0.2782,0.2782,0.2780,0.2786,0.2792,0.2798,0.2804,0.2810,0.2815,0.2821,0.2827,0.2833,0.2839,3
1,2015,1478,140.77,-0.0271,-0.0254,-0.0231,-0.0206,-0.0183,-0.0163,-0.0151,-0.0150,-0.0155,-0.0162,-0.0169,-0.0176,-0.0183,-0.0191,-0.0199,-0.0206,-0.0214,-0.0220,-0.0227,-0.0232,-0.0237,-0.0240,-0.0243,-0.0245,-0.0247,-0.0249,-0.0250,-0.0251,-0.0252,-0.0253,-0.0254,-0.0254,-0.0255,-0.0256,-0.0256,-0.0257,-0.0258,-0.0259,-0.0260,-0.0262,-0.0263,-0.0265,-0.0268,-0.0270,-0.0273,-0.0278,-0.0285,-0.0294,-0.0304,-0.0316,-3.290001e-02,-0.0343,-0.0357,-0.0370,-0.0384,-0.0397,-0.0409,-0.0419,-0.0428,-0.0435,-0.0440,-0.0440,-0.0434,-0.0424,-0.0412,-0.0400,-0.0388,-0.0380,-0.0376,-0.0377,-0.0382,-0.0391,-0.0402,-0.0416,-0.0431,-0.0447,-0.0463,-0.0478,-0.0492,-0.0504,-0.0514,-0.0520,-0.0523,-0.0521,-0.0513,-0.0500,-0.0388,-0.0276,-0.0164,-0.0052,0.0060,0.0171,0.0283,0.0395,5.070001e-02,0.0619,0.0731,0.0843,0.0955,0.1067,0.1179,0.1290,0.1402,0.1514,0.1626,0.1738,0.185

In [44]:
class_count_df = df.groupby(TARGET).count()

In [45]:
classes=class_count_df['0']

In [46]:
type(classes)

pandas.core.series.Series

In [47]:
new_weights=[]
for i in classes:
    new_weights.append((classes.sum()) / (2.0 * i))

In [48]:
class_weights=torch.FloatTensor(new_weights)

In [49]:
class_weights

tensor([3.9417e+01, 2.8959e+01, 4.1982e+00, 1.0907e+00, 1.0750e+01, 1.0136e+02,
        5.7683e+00, 1.6694e+01, 7.6290e+00, 2.3650e+01, 3.1533e+01, 1.8192e+01,
        4.3000e+01, 1.6310e+01, 2.0271e+02, 2.4466e+01, 6.7571e+01, 7.0950e+02,
        7.0950e+02, 2.0271e+02, 5.6760e+01, 1.7738e+02, 2.3650e+02, 2.3650e+02,
        7.0950e+02, 1.4190e+03, 1.4190e+03, 7.0950e+02])

In [50]:
metrics = F1ScoreMulti(average='weighted')

In [51]:
loss_func = BCEWithLogitsLossFlat(weight=class_weights)

In [52]:
learn = tabular_learner(dls,layers=[586,1024], loss_func=loss_func, metrics=metrics)

In [56]:
learn.model

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(6, 4)
    (1): Embedding(1525, 97)
    (2): Embedding(1584, 99)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(290, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(490, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=490, out_features=586, bias=False)
      (2): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(586, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=586, out_features=1024, bias=False)
      (2): ReLU(inplace=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=1024, out_features=27, bias=True)
    )
  )
)

In [55]:
learn.fit(1)

ValueError: Target size (torch.Size([64])) must be the same as input size (torch.Size([1728]))